In [2]:
import os
import PyPDF2

In [4]:
#Function to Load pdf documents
def load_pdf_doc(directory):
    docs=[]
    file_names=[] 
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path=os.path.join(directory,filename)
            with open(file_path,'rb') as file:
                reader=PyPDF2.PdfReader(file)
                text=''
                for page in range(len(reader.pages)):
                    text+= reader.pages[page].extract_text()
                docs.append(text)
                file_names.append(filename)
    return docs,file_names
pdf_directory=r'brochures'
docs, file_names=load_pdf_doc(pdf_directory)

In [7]:
#Intialize the Vectorization and fit the documents
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
vectorizer=TfidfVectorizer()
doc_vectors= vectorizer.fit_transform(docs)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [11]:
def retrieve_relevent_doc(query, doc_vectors, vectorizer,docs,file_names):
    query_vector=vectorizer.transform([query])
    similarities=cosine_similarity(query_vector,doc_vectors).flatten()
    most_similar_doc_index=np.argmax(similarities)
    return docs[most_similar_doc_index], file_names[most_similar_doc_index]

In [16]:
#Load Pre trained model and tokenizer

from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [18]:

tokenizer =GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')

def generate_response(query,context):
#Combine the context and query into a single input for the model
    input_text = f"{context} [SEP] {query}"
    input_ids =tokenizer.encode(input_text, return_tensors='pt')

#Generate the output
    output=model.generate(input_ids, max_length=50, num_return_sequences=1)
    response= tokenizer.decode(output[0], skip_special_tokens=True)
    return response

#RAG Pipeline

def rag_pipeline(query):
    #Step 1: Retrieving the most relevant document
    context, file_name = retrieve_relevent_doc(query, doc_vectors,vectorizer, docs, file_names)
    print(f"Retrieved Document: {context[:200]}....\n")  # print first 200 characters of the document
    print(f"Document Source:{file_name}\n")

    # Step 2: Generate a response using the retrieved document as context
    response=generate_response(query,context)
    return response

query =input("Hi. I am your Travel assistant. How can I help you?")
response= rag_pipeline(query)
print(f"Generated Response:{response}")




C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Retrieved Document: Margie’s Travel Presents…  
London  
London is the capital and 
most populous city of 
England and the United 
Kingdom. Standing on the 
River Thames in the south 
east of the island of Great 
Britain....

Document Source:London Brochure.pdf



ValueError: Input length of input_ids is 399, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.